In [ ]:

# notebook to analyze webgpu shaders

In [ ]:

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

plt.figure(dpi=1200);
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 120)

In [ ]:
def json_to_df(profile_path, idx):
    entries = []
    seen = 0
    
    with open(profile_path, "r") as f:
        data = json.load(f)

    if type(data) == dict:
        data = data['traceEvents']

    first_name = None

    for item in data:
        dur = item.get("dur")
        if dur is None:
            continue
        cat = item.get("cat")
        if cat not in ["Api"]:
            continue
        arg = item.get('args')
        if not arg:
            continue
        ts = item.get("ts")
        name = item['name']
        if not first_name:
            first_name = name
        if first_name == name:
            seen += 1
        if seen != idx:              
            continue
        shapes = arg.get('shapes')
        cols = name.split("&")
        
        e = {
            "ts": ts, "name": cols[0], "op_type": cols[1], "shader": cols[2], "key": name, "dur": dur, "shape": shapes,
        }
        entries.append(e)
    df = pd.DataFrame([f for f in entries])
    df['count'] = 1
    return df


def show(csv, idx, shape=False, filter=None):
    df = json_to_df(csv, idx)
    ts_total = df['ts'].iloc[-1] - df['ts'].iloc[0]
    df2 = df[['dur', 'count']].sum()
    df['pct'] = (100 * df['dur'] / df2['dur'])
    top = 50
    digits = 2
    key = ["shader"]

    if filter:
        df = df[df['shader'].str.contains(filter)]

    if shape:
        key.append("shape")
    # key = ["shader"]
    fields = key.copy()
    fields.extend(["dur", "pct", "count"]) 
    df1 = df[fields].groupby(key).sum()
    df1 = df1.sort_values(by=['dur'], ascending=False) # [:top]
    df1['avg'] = df1['dur'] / df1['count']
    df1['csum_dur'] = df1['dur'].cumsum()
    df1['csum'] = df1['pct'].cumsum()
    df1['pct_total'] = 100 * df1['csum_dur'] / ts_total
    print(f"\n-- Top shader by total runtime, {csv}\n")
    print(df1.round(digits).to_string(index=True))
    return df1



In [ ]:
df_samsung_default= show("C:\dev\perf_results\samsung\samsung_default_yolo11n_2025-04-25_16-53-53.json", 1, True, "MatMul").sort_index()
df_samsung_workgroup = show("C:\dev\perf_results\samsung\samsung_default_yolo11n_4_4_1_4_4_1_16_2025-04-25_16-55-18.json", 1, True, "MatMul").sort_index()

comparison_df =  df_samsung_default[['avg']].join(
    df_samsung_workgroup[['avg']],
    lsuffix='_default',
    rsuffix='_workgroup'
)
comparison_df['ratio'] = comparison_df['avg_default'] / comparison_df['avg_workgroup']

comparison_df = comparison_df.sort_values(by='ratio', ascending=False)

print(comparison_df.round(2))

In [ ]:
df_pixel_default= show("C:\dev\perf_results\pixel\perf_results\yolo11n_pixel_fp32_default_2025-04-25_15-52-17.json", 1, True, "MatMul").sort_index()
df_pixel_workgroup = show("C:\dev\perf_results\pixel\perf_results\yolo11n_pixel_fp32_4_4_1_4_4_1_16_2025-04-25_15-52-02.json", 1, True, "MatMul").sort_index()

comparison_df =  df_pixel_default[['avg']].join(
    df_pixel_workgroup[['avg']],
    lsuffix='_default',
    rsuffix='_workgroup'
)
comparison_df['ratio'] = comparison_df['avg_default'] / comparison_df['avg_workgroup']

comparison_df = comparison_df.sort_values(by='ratio', ascending=False)

print(comparison_df.round(2))


In [ ]:
df_samsung = show("C:/dev/perf_results/samsung/yolo11n_fp32_samsung_webgpu_2025-04-22_14-56-25.json", 1, True, "Transpose").sort_index()

df_pixel = show("C:/dev/perf_results/pixel/perf_results/yolo11n_fp32_pixel_webgpu_2025-04-22_14-56-24.json", 1, True, "Transpose").sort_index()

# calculate average duration ratio samsung vs pixel for each matmul shader shape
# Join the two dataframes on their index (shader + shape)
comparison_df = df_samsung[['avg']].join(
    df_pixel[['avg']],
    lsuffix='_samsung',
    rsuffix='_pixel'
)

# Compute the ratio of avg duration
comparison_df['ratio_pixel_to_samsung'] = comparison_df['avg_pixel'] / comparison_df['avg_samsung']

# Optional: sort by ratio to see which shape is worse on Pixel
comparison_df = comparison_df.sort_values(by='ratio_pixel_to_samsung', ascending=False)

# Display the result
print(comparison_df.round(2))

In [ ]:

show("./webgpu_matmul_2025-04-10_14-38-38.json", 1)
show("./webgpu_matmul_2025-04-10_14-38-38.json", 2)


In [ ]:
show("./yolo11n_webgpu_2025-04-17_14-25-43.json", 1)
show("./yolo_webgpu_fp32_2025-04-17_17-07-40.json", 1)

In [ ]:
show("./resnet_webgpu_2025-04-17_16-45-24.json", 1)

In [ ]:
show("./yolo11n_webgpu_2025-04-17_14-25-43.json", 1)
show("./pixel_webgpu_yolo_fp16_2025-04-17_18-10-34.json", 3)

In [ ]:
show("C:/dev/perf_results/samsung/yolo11n_fp32_samsung_webgpu_2025-04-22_14-56-25.json", 1, False)

show("C:\dev\perf_results\pixel\perf_results\yolo11n_fp32_pixel_webgpu_2025-04-22_14-56-24.json", 1, False)

